# Rendering Plant Integrations for MiFlora Sensors from CSV

This notebook will focus on the generating the Plant Integration specifically for the Miflora Plant Sensors that were used with the ESPHome Configuration Generation in this [project](./AutoGenerate%20ESP32%20Configurations.ipynb)

Date of Project: July 24 2022
Software and Devices Used: 
- Home Assistant 2022.7
- [Miflora Sensor](https://geni.us/HKFMiflora) Xiaomi BLE Plant Sensor
- [ESP32](https://geni.us/HKGesp32)-Wroom-RevD

Further notebooks will be created to document the other portions of this project which will be available on my Github repositoriy located here.

https://github.com/netmanchris/HassPlants



# Miflora BLE Plant Sensor + ESP32 Config Generation for Home Assistant

Like most things, what was easy at low volume can start to show challenges when reaching certain scales. 

This project is intended to help automate the creation of my home assistant based Plant monitoring setup. This setup uses a combination of the Xiaomi [Miflora Sensor](https://geni.us/HKFMiflora) BLE Plant sensors, [ESP32](https://geni.us/HKGesp32) wroom RevD boards, and Home Assistant. 

This has been a work in progress and will likely change as I experiment. 

## WHY

### Why part 1

I killed plants. I'd like my plants to not die.

Seriously. I made the double mistake of over-fertilizing a bunch of plants AND doing it when the soil was really dry. From what I understand, when plants are overly dry, the roots suck up as much water as they can when they finally have access to water. The combination of the two means that the roots suck up far too much fertilizer in the water and burn out the roots and plant completely.  Leaving you something looking a little like this.

![image](./images/dead_spider.jpg)

### Why part 2

Why not? Home Automation is supposed to help us and make our lives easier. This project will allow me to know exactly when any given plant in my home needs water and/or fertilization. I'm not wandering around the house touching plants to see if the it's dry or using a plant monitoring spike from plant to plant to check the soil fertilization. 


When I started this project, it was possible to integrate the Miflora Sensor suing the native BLE Home Assistant Integration, but as of 2022.7 release, the bluethooth libraries that Home Assistant used are no longer usable which will break existing [Miflora Sensor](https://geni.us/HKFMiflora) integrations using the BLE, and makes BLE no longer an option moving forward. 

![image](./images/20227BLEBroke.png)

So moving forward, the ESPHome based is the easiest of the available options.

In addition to the "It's the easiest way" reason, the other major reason to use the ESPHome based integration is that it allows you to esaily overcome the distance limitations that come along with Bluetooth as a networking technology. 

### Simple Solution

For users wanting to leverage the simple ESPHome based solution, this can be easily done manually and allows you to deploy this solution in 

- Scan the BLE mac-address of the [Miflora Sensor](https://geni.us/HKFMiflora) using an android-based device (iOS devices don't show the mac-address at the time of this writting)
- Connect the [ESP32](https://geni.us/HKGesp32) to the ESPHome interface in Home Assistant and add the [Miflora Sensor](https://geni.us/HKFMiflora) specific configuration using the mac-address detected in step 1.
- Manually configurat the YAML definition for the Plant integration using the Sensors entities assigned in step 2.
- Add the new plant entities to dashboards

![image](./images/singlesensor.png)

### Scaled Solution

When the same four steps above need to be performed at larger scales, it quickly became obvious to me that I needed to do something to automate these steps. Although this process can be useful to automate the creation of the various configuration files when dealing with just a few sensors, the majority of the work of this project was performed to deal with the a setup with the following attributes

- 1 x home
- 7 x rooms
- 9 x Esp32-Wroom Rev D
- ~80 x [Miflora Sensor](https://geni.us/HKFMiflora)

![image](./images/multiplesensor.png)







# Description of this section

In this part of the section, we're going to use a combination of the plant_name and mac_address values in the plant_worksheet.csv file to generate the sensor portion of the ESPHome configuration file. 

![image](./images/miflora_esp_plant.png)

To be clear, this works because we are using the same data to generate the plant integration YAML files that we used to create the ESPHome Configuration files for our [Miflora Sensor](https://geni.us/HKFMiflora) and [ESP32](https://geni.us/HKGesp32). This means that the entity_id names in Home Assistant will be predictable which is why this works. 

The sensor names generated by this step are used in future parts of this project extensivly. Having a consistent, predictable naming convention means that we can generate additional parts of the home assistant configuration such as
- plant integrations
- dashboards
- automations
- scripts
- I'm sure there's more! 

## In a nutshel

We're going to automatically create a [plant monitor](https://www.home-assistant.io/integrations/plant/) for every plant in the worksheet CSV file that has a Miflora sensor and automatically configure the same sensor names that were created in the ESPHome configuration. 




# CSV File Formating and Plant Integration Variable Generation

There are currently two files which are required as data inputs for this project

- plants_worksheet.csv is the file where the plants to include in the generation of the input_helpers will be stored
- plants_attributes.csv is the file where the specific min/max values for different species of plants are stored






# Plants Worksheet CSV

The Plants worksheet is a CSV which contains all of the information required to generate the [ESP32](https://geni.us/HKGesp32) configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

A sample of the plant worksheet can be located [here](./data/plants_worksheet_sample.csv).

Each row of this worksheet represents a single plant with the following attributes\

- name: Unique Value. One per plant. type: String
- plant_type: One per plant. Must match values in the *plant_attributes.csv* worksheet type: String
- room: Grouping mechanism. Plants with the same value will be automatically grouped together.
- plant_order: {Future Use} Plan is to use this to help organize and sort plants easier. type: Integer
- sensor: Unique Value. One per sensor. type: Integer
- sensor_number: Unique Value. One per sensor. This will be appended in the ESP config as the unique portion of the auto-generated sensor entity.  type: String
- sensor_mac: the 12 digit mac-address in the format XX:XX:XX:XX:XX:XX example C4:7C:8D:6D:8D:F6 type: String
- esp_parent: {FUTURE USE} Intention is to use this to create a permanent binding between a specific plant sensor and a specific ESP32 device to prevent naming conflicts when dealing with larger rooms ( > 13 plants per room ) type: String 
- esp_current: Unused. Documentation purposes only. type: String
- room_picture: {FUTURE USE} Intention is to use this to bind specific plants to specific room pictures for building - out dashboards that require multiple camera angles. type: String
- top: Numerical value used a the X coodinate for plant placement for the room picture generation. type: Integer
- left: Numerical value used a the Y coodinate for plant placement for the room picture generation. type: Integer
- zone: Used to designate a specific zone in a room for use with the flash_light portion of this project.
- flash_light: {to be removed in the future} full name of the script entitiy used to flash the lights where a specific plant is located. type: String



## Plants Attributes CSV

The Plants worksheet is a CSV which contains all of the information required to generate the ESP32 configurations, plant integration configurations, as well as the dashboards for both desktop/tablet and mobile formats.

The plant worksheet can be located [here](./data/plant_attrributes.csv). Further documentation on the exact values below can be found on the Home Assistant Plant integration page [here](https://www.home-assistant.io/integrations/plant/)

Each row of this worksheet represents a single plant with the following attributes


- plant: Unique Value. One per plant species. type: String
- species: Unique value. One per plant species. type: String
- min_moisture: minimum moisture value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer between 1 and 100
- max_moisture: maximum moisture value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer between 1 and 100
- min_conductivity: minimum soil fertilization level value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_conductivity: maximum soil fertilization level value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- min_temperature: minimum temperature value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_temperature: maximum temperature value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- min_brightness: minimum brightness value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- max_brightness: maximum brightness value value reported by linked sensor where Home Assistant should trigger an alarm condition. Type: Integer
- check_days: time interval (in days) used when checking min_brightness.

**Note** The values in the [plant attributes](./data/plant_attrributes.csv) can be adjusted to whatever makes sense in your environment. As long as the *species* attribute in the plant attributes file and *plant_type* attribute in the plant_worksheet.csv file match, the plant monitor will be filled in correctly. 

For the values currently in the [plant attributes](./data/plant_attrributes.csv), I've pulled this directly out of the MiHome app (FlowerCare) which has a large database of plants and values to search through if the specific type of plant you're looking for isn't already showing in the file, this is a great place to look.

For instance, for the Peace Lily...

![image](./images/mi_app1.PNG)

![image](./images/mi_app2.PNG)

![image](./images/mi_app3.PNG)



# Miflora Plant Integration Jinja Template

The Goal of this project will be to do the following

- Identify and Filter out only the plants in the plants_worksheet.csv file which are listed as having MiFlora sensors. 
- Write a set of files to disk for Plant integration Definitions. One file per room which should contain all plants in that room. 

Bottom line; a complete plant integration entity will be created for each [Miflora Sensor](https://geni.us/HKFMiflora) in the spreadsheet and placed in a file labeled with the title of the room where that plant is located.  

The full plant monitor template can be seen [here](./templates/room_plants.j2)

The template is split into 2 sections

1. Bind Sensors to this Plant - This should automatically generate the plant definition including the appropriate binding to the right sensor entity for the specific [Miflora Sensor](https://geni.us/HKFMiflora).

2. Fill in Plant Species Specific Values - This should automatically generate the plant definition included the appropriate values based on the plants species from the plants_attributes.csv file above. 

```
{% for plant in plants %}{% if plant.sensor_model == 'Miflora' %}
{{plant.name}}:
    sensors:
      moisture: sensor.plant_{{plant.sensor}}_moisture
      temperature: sensor.plant_{{plant.sensor}}_temperature
      conductivity: sensor.plant_{{plant.sensor}}_soil_conductivity
      brightness: sensor.plant_{{plant.sensor}}_illuminance{% for plant_type in plant_types%}{% if plant.plant_type == plant_type.species%}
    min_moisture: {{plant_type.min_moisture}}
    max_moisture: {{plant_type.max_moisture}}
    min_conductivity: {{plant_type.min_conductivity}}
    max_conductivity: {{plant_type.max_conductivity}}
    min_temperature: {{plant_type.min_temperature}}
    max_temperature: {{plant_type.max_temperature}}
    #min_brightness: {{plant_type.min_brightness}}
    max_brightness: {{plant_type.max_brightness}}{% endif%}{% endfor %}
    check_days: 3

    {% endif %}{% endfor %}
```




# Generating the Configuration Files

In this section, we'll go through the python code used to generate the actual configurations




In [1]:
import pandas as pd
from jinja2 import Template
import os

# Load Required Data Files CSV

In this section, we will load the plants_worksheet and plant_attributes CSV files which will be used to create the Home Assistant Plant Monitors and bind the [Miflora Sensor](https://geni.us/HKFMiflora) to the appropriate plant attribute.

There are two files needed here.

1. plants_worksheet.csv which includes the various plants, sensors, room locations, corresponding script to flash lights.

2. plants_attributes.csv which includes the specific min/max values for the various sensors to be used with the plant integration








In [15]:
plants_df = pd.read_csv('data/plants_worksheet.csv', index_col=0)
#plants_df = pd.read_csv('data/plants_worksheet_sample.csv', index_col=0)
plant_attribs = pd.read_csv('data/plant_attrributes.csv', index_col=0).to_dict('records')

manual_sensors = pd.read_csv('data/room_sensors.csv', index_col=0).to_dict('records')

## Create Rooms List

In this step, we will create a list of the room names defined in the CSV file. Although not strictly necessary, defining the room names logically is useful in that plant monitor configurations. Plant Monitor integration YAML definitions for each individual room will be automatically generated through this code. Spending some time thinking about how you want to organize your plants dashboards is a very, very good idea.



In [4]:
rooms_list = list(plants_df.room.unique())
rooms_list

['kitchen',
 'living_room',
 'dining_room',
 'home_office',
 'downstairs_bathroom',
 'master_bedroom',
 'games_room',
 'boys_bedroom',
 nan]

## Loading the Plants Integration Jinja2 Templates

In this step, we load the plants specific templates from the templates folder and load it as a jinja2 template.



In [26]:
with open('templates/room_plants.j2') as f:
    room_template = f.read()
j2_plantroom_template = Template(room_template)

## Rendering Plant Configurations

We will now render the Plant Configurations where one file will be generated for each room




Note: As a prerequisite, the directory structure has to preexist to be able to properly save the file configurations to disk. 

Make sure that the folder "plants" exists under the output folder.






In [27]:
# Includes the manual plants
# Includes the LilyGo plants

for room in rooms_list:
    if type(room) is str:  
        room = [room]
        room_df = plants_df[plants_df.room.isin(room)]
        #print (len(room_df))
        if len(room_df) > 0:
            #print (room)
            #room_df = room_df[room_df.sensor_model.eq('Miflora')]
            room_dict = room_df.to_dict('records')
            #print (len(room_dict))

            # OUTPUT A SINGLE FILE FOR EACH ROOM WITH ALL THE MiFLORA PLANTS INTEGRATION YAML WITH THE PROPER MIFLORA SENSOR NAMES
            with open('output/plants/'+room_dict[0]['room']+'.txt', 'w') as f: 
                f.write(j2_plantroom_template.render(plants=room_dict, plant_types=plant_attribs,manual_sensors = manual_sensors))

# Sample Output

The code above should generate a single file per room which contains all of the required plants.

The following is the a sample of the file labeled "dining_room.txt".

As you can see, the plant definition is automatically using the sensor name which matches that auto-generated in the [Auto Generation of ESP32 Configurations for use with ESP Home](./AutoGenerate%20ESP32%20Configurations.ipynb)section of this project

```
DR_Dracaena_1:
    sensors:
      moisture: sensor.plant_s7_moisture
      temperature: sensor.plant_s7_temperature
      conductivity: sensor.plant_s7_soil_conductivity
      brightness: sensor.plant_s7_illuminance
    min_moisture: 15
    max_moisture: 60
    min_conductivity: 200
    max_conductivity: 1500
    min_temperature: 10
    max_temperature: 35
    #min_brightness: 2000
    max_brightness: 50000
    check_days: 3

```

The configuration should be cut and paste into a file labeled %room_name%.yaml in the appropriate folder

```
config
   ->plants
```
       
This should also require adding a line to the configuration.yaml file to make sure that the structure is included in the processing of the overall configuration.


```plant: !include_dir_merge_named plants```

![image](./images/config_plants_yaml.png)


Once you reload your configuration or restart your home assistant instance, you should be able to verify that the plants are shown as follows


Navigate to **Settings > Devices & Services > entities** to validate that the plants were created as desired.

![image](./images/hass_plant_entities.png)

By click on a specific entity you can validate the current status and configuration 

![image](./images/plant_entity_1.png)

By clickig on the attributes link, you can also validate the specific sensors that any individual plant.

![image](./images/plant_entity_2.png)


# End Goal

The entire purpose of this is to bind a specific setting to a specific plant so that you are now able to use this device in your home assistant dashboards.

Now that you have bound all the sensors to a specific plant, you'll be able to use the plant card to show the current status of any individual plant


![image](./images/plant_card_1.png)